# Clustering Consulting Project 

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('cluster_proj').getOrCreate()

In [3]:
from pyspark.ml.clustering import KMeans

# Loads data.
dataset = spark.read.csv("/FileStore/tables/hack_data.csv",header=True,inferSchema=True)

In [4]:
dataset.show()

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted| Location|WPM_Typing_Speed|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
 8.0| 391.09| 1| 2.96| 7.0| Slovenia| 72.37|
 20.0| 720.99| 0| 3.04| 9.0|British Virgin Is...| 69.08|
 31.0| 356.32| 1| 3.71| 8.0| Tokelau| 70.58|
 2.0| 228.08| 1| 2.48| 8.0| Bolivia| 70.8|
 20.0| 408.5| 0| 3.57| 8.0| Iraq| 71.28|
 1.0| 390.69| 1| 2.79| 9.0| Marshall Islands| 71.57|
 18.0| 342.97| 1| 5.1| 7.0| Georgia| 72.32|
 22.0| 101.61| 1| 3.03| 7.0| Timor-Leste| 72.03|
 15.0| 275.53| 1| 3.53| 8.0|Palestinian Terri...| 70.17|
 12.0| 424.83| 1| 2.53| 8.0| Bangladesh| 69.99|
 15.0| 249.09| 1| 3.39| 9.0|Northern Mariana ...| 70.77|
 32.0| 242.48| 0| 4.24| 8.0| Zimbabwe| 67.93|
 23.0| 514.54| 0| 3.18| 8.0| Isle of Man| 68.56|
 9.0| 284.77| 0| 3.12| 9.0|Sao Tome and Prin...| 70.82|
 27.0| 779.25| 1| 2.37| 8.0| Greece| 72.73|
 12.0| 307.31| 1| 3.22| 7.0| Solomon Islands| 67.95|
 21.0| 355.94| 1| 2.0| 7.0| Guinea-Bissau| 72.0|
 10.0| 372.65| 0| 3.33| 7.0| Burkina Faso| 69.19|
 20.0| 347.23| 1| 2.33| 7.0| Mongolia| 70.41|
 22.0| 456.57| 0| 1.52| 8.0| Nigeria| 69.35|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
only showing top 20 rows

In [5]:
dataset.printSchema()

root
-- Session_Connection_Time: double (nullable = true)
-- Bytes Transferred: double (nullable = true)
-- Kali_Trace_Used: integer (nullable = true)
-- Servers_Corrupted: double (nullable = true)
-- Pages_Corrupted: double (nullable = true)
-- Location: string (nullable = true)
-- WPM_Typing_Speed: double (nullable = true)

In [6]:
final_data = dataset.drop('Location')

final_data.show(5)

+-----------------------+-----------------+---------------+-----------------+---------------+----------------+
Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|WPM_Typing_Speed|
+-----------------------+-----------------+---------------+-----------------+---------------+----------------+
 8.0| 391.09| 1| 2.96| 7.0| 72.37|
 20.0| 720.99| 0| 3.04| 9.0| 69.08|
 31.0| 356.32| 1| 3.71| 8.0| 70.58|
 2.0| 228.08| 1| 2.48| 8.0| 70.8|
 20.0| 408.5| 0| 3.57| 8.0| 71.28|
+-----------------------+-----------------+---------------+-----------------+---------------+----------------+
only showing top 5 rows

In [7]:
from pyspark.ml.feature import VectorAssembler

In [8]:
final_data.columns

Out[82]: ['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'WPM_Typing_Speed']

In [9]:
df02 = VectorAssembler(inputCols = final_data.columns, outputCol = 'features').transform(final_data)

In [10]:
from pyspark.ml.feature import StandardScaler

In [11]:
scaled_data = StandardScaler(inputCol = 'features', outputCol = 'scaledFeatures', withMean=False, withStd=True).fit(df02).transform(df02)

In [12]:
# Trains a k-means model.
kmeans01 = KMeans(featuresCol='scaledFeatures',k=2)
model01 = kmeans01.fit(scaled_data)

kmeans02 = KMeans(featuresCol='scaledFeatures',k=3)
model02 = kmeans02.fit(scaled_data)



In [13]:
# trainingCost = Sum of squared distances to the nearest centroid for all points in the training dataset. This is equivalent to sklearn’s inertia.


print('2 centroids - Sum of Squared distances to the nearest centroid = {}'.format(model01.summary.trainingCost))

print('3 centrois - Sum of Squared distances to the nearest centroid = {}'.format(model02.summary.trainingCost))


2 centroids - Sum of Squared distances to the nearest centroid = 601.7707512676691
3 centrois - Sum of Squared distances to the nearest centroid = 434.1492898715821

In [14]:
model02.transform(scaled_data).select('prediction').show()

+----------+
prediction|
+----------+
 1|
 2|
 1|
 1|
 2|
 1|
 1|
 1|
 1|
 1|
 1|
 2|
 2|
 2|
 1|
 1|
 1|
 2|
 1|
 2|
+----------+
only showing top 20 rows